# running EC2 commands via boto3

Boto3 documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ssm.html#SSM.Client.send_command

Couple of gotchas while POCing:
  * required install of SSM on EC2: https://docs.aws.amazon.com/systems-manager/latest/userguide/agent-install-al2.html
  * required IAM permissions for SSM: https://docs.aws.amazon.com/systems-manager/latest/userguide/setup-instance-profile.html
  * seemed to require a reboot

In [ ]:
import time
import boto3

In [ ]:
# init boto3 SSM client
client = boto3.client('ssm', region_name='us-east-1')

In [ ]:
# send hello world
results = client.send_command(
    InstanceIds=["i-07a0c73b0ceb6ce87"],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": ["echo 'hello world'"]},
)
print(results)

In [ ]:
results

In [ ]:
# get results of command (noting this won't be needed mostly)
results = client.get_command_invocation(
    InstanceId="i-07a0c73b0ceb6ce87",
    CommandId="4c43e368-523f-4c06-a35e-e4a4de27b857"
)

In [ ]:
results

## notes on using SSM for issuing Kafka CLI commands on EC2 

First pass at running Kafka CLI command failed due to following permission error:

```
An error occurred (AccessDeniedException) when calling the DescribeCluster operation: User: arn:aws:sts::007385147054:assumed-role/SSMInstanceProfile/i-07a0c73b0ceb6ce87 is not authorized to perform: kafka:DescribeCluster on resource: arn:aws:kafka:us-east-1:007385147054:cluster/poc-cluster-1/7a7a5d89-0d30-4a3c-94ab-b6c4f18c0fc0-21\nfailed to run commands: exit status 255
```

This was resolved by granting more IAM permissions to the profile -- `arn:aws:iam::007385147054:role/SSMInstanceProfile` -- that was granted to the ECS machine that allowed SSM commands. 

In [ ]:
# run Kafka CLI command; polling for results
# prepare command string
cmd = 'aws kafka describe-cluster --region us-east-1 --cluster-arn "arn:aws:kafka:us-east-1:007385147054:cluster/poc-cluster-1/7a7a5d89-0d30-4a3c-94ab-b6c4f18c0fc0-21"'

# send command
cmd_send_results = client.send_command(
    InstanceIds=["i-07a0c73b0ceb6ce87"],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": [cmd]},
)

# wait 2 seconds (improve w/ polling?)
time.sleep(2)

# retrieve results of SSM command and print stdout
cmd_get_results = client.get_command_invocation(
    InstanceId="i-07a0c73b0ceb6ce87",
    CommandId=cmd_send_results['Command']['CommandId']
)
print(cmd_get_results['StandardOutputContent'])